# Import thư viện

In [1]:
import os

from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from utils import *

set_config(display='diagram')

# Đọc dữ liệu

In [2]:
path_to_data = '../data'
path_track = os.path.join(path_to_data, '20210824_212829_tracks.tsv')
path_playlist = os.path.join(path_to_data, '20210824_212829_playlists.tsv')
path_audio = os.path.join(path_to_data, '20210824_212829_audios.tsv')
path_artist = os.path.join(path_to_data, '20210824_212829_artists.tsv')

In [3]:
pd_track = pd.read_csv(path_track, sep='\t')
pd_audio = pd.read_csv(path_audio, sep='\t')
pd_playlist = pd.read_csv(path_playlist, sep='\t')
pd_artist = pd.read_csv(path_artist, sep='\t')

In [4]:
pd_audio

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,5HCyWlXZPP0y6Gqq8TgA20,0.591,0.764,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
1,27NovPIUIRrOZoCHxABJwK,0.736,0.704,3,-7.409,0,0.0615,0.0203,0.000000,0.0501,0.894,149.995,4
2,3Vi5XqYrmQgOYBajMWSvCi,0.664,0.609,1,-6.509,1,0.0707,0.3040,0.000000,0.0926,0.194,130.041,4
3,6PQ88X9TkUIAUIZJHW2upE,0.808,0.897,11,-3.712,0,0.0348,0.0469,0.000031,0.3640,0.591,126.026,4
4,3Wrjm47oTz2sjIgck11l5e,0.714,0.800,11,-4.808,0,0.0504,0.1270,0.000000,0.3590,0.589,134.002,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69866,3Zuf70897YkrVRAsrBMMSF,0.751,0.753,0,-8.198,1,0.0935,0.2270,0.000784,0.2090,0.774,106.976,4
69867,1MDoll6jK4rrk2BcFRP5i7,0.471,0.431,5,-6.129,0,0.0342,0.3290,0.000000,0.0854,0.289,157.980,4
69868,7DqTaelFf846rl1CHmyfOW,0.506,0.433,2,-14.183,0,0.0345,0.8420,0.031100,0.1120,0.641,83.222,4
69869,0ygTmpa6uSotkBkTiwcMZ4,0.493,0.590,0,-5.814,0,0.0392,0.3390,0.000000,0.1210,0.502,169.908,4


In [5]:
pd_audio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69871 entries, 0 to 69870
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                69871 non-null  object 
 1   danceability      69871 non-null  float64
 2   energy            69871 non-null  float64
 3   key               69871 non-null  int64  
 4   loudness          69871 non-null  float64
 5   mode              69871 non-null  int64  
 6   speechiness       69871 non-null  float64
 7   acousticness      69871 non-null  float64
 8   instrumentalness  69871 non-null  float64
 9   liveness          69871 non-null  float64
 10  valence           69871 non-null  float64
 11  tempo             69871 non-null  float64
 12  time_signature    69871 non-null  int64  
dtypes: float64(9), int64(3), object(1)
memory usage: 6.9+ MB


In [6]:
pd_track

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity
0,37i9dQZF1DXcBWIGoYBM5M,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100
1,37i9dQZF1DXcBWIGoYBM5M,27NovPIUIRrOZoCHxABJwK,INDUSTRY BABY (feat. Jack Harlow),212000,"['7jVv8c5Fj3E9VhNjxT4snq', '2LIk90788K0zvyj2JJ...","['Lil Nas X', 'Jack Harlow']",622NFw5Yk0OReMJ2XWcXUh,INDUSTRY BABY (feat. Jack Harlow),97
2,37i9dQZF1DXcBWIGoYBM5M,3Vi5XqYrmQgOYBajMWSvCi,Need To Know,210560,['5cj0lLjcoR7YOSnhnX0Po5'],['Doja Cat'],1nAQbHeOWTfQzbOoFrvndW,Planet Her,92
3,37i9dQZF1DXcBWIGoYBM5M,6PQ88X9TkUIAUIZJHW2upE,Bad Habits,231041,['6eUKZXaKkcviH0Ku9w2n3V'],['Ed Sheeran'],01NhUvviMytvV12pmJiDZH,Bad Habits,98
4,37i9dQZF1DXcBWIGoYBM5M,3Wrjm47oTz2sjIgck11l5e,Beggin',211560,['0lAWpj5szCSwM4rUMHYmrr'],['Måneskin'],2qJw6w5XwQO0PQlSWPu7Tw,Chosen,99
...,...,...,...,...,...,...,...,...,...
88814,4lgseztVwmKQ8MNETPVIny,5kqIPrATaCc2LqxVWzQGbk,7 Years,237300,['25u4wHJWxCA9vO0CzxAbK7'],['Lukas Graham'],4rFrdkSWs0dtj0rWPzOk1v,Lukas Graham,79
88815,4lgseztVwmKQ8MNETPVIny,3FCto7hnn1shUyZL42YgfO,Piano Man,336093,['6zFYqv1mOsgBRQbae3JJ9e'],['Billy Joel'],7r36rel1M4gyBavfcJP6Yz,The Essential Billy Joel,66
88816,4lgseztVwmKQ8MNETPVIny,3XVBdLihbNbxUwZosxcGuJ,If I Ain't Got You,228706,['3DiDSECUqqY1AuBP8qtaIa'],['Alicia Keys'],6TqRKHLjDu5QZuC8u5Woij,The Diary Of Alicia Keys,78
88817,4lgseztVwmKQ8MNETPVIny,6QPKYGnAW9QozVz2dSWqRg,Someone Like You,285240,['4dpARuHxo51G3z768sgnrY'],['Adele'],7n3QJc7TBOxXtlYh4Ssll8,21,13


In [7]:
pd_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88819 entries, 0 to 88818
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   playlist_id     88819 non-null  object
 1   track_id        88819 non-null  object
 2   track_name      88812 non-null  object
 3   track_duration  88819 non-null  int64 
 4   artist_ids      88819 non-null  object
 5   artist_names    88819 non-null  object
 6   album_id        88819 non-null  object
 7   album_name      88811 non-null  object
 8   popularity      88819 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 6.1+ MB


In [8]:
pd_playlist

,playlist_id,playlist_name,playlist_description,playlist_num_tracks,playlist_num_followers
0,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
1,37i9dQZF1DX0XUsuxWHRQd,RapCaviar,"New music from Young Thug, Trippie Redd, Rod W...",50,14030878
2,37i9dQZF1DX1lVhptIYRda,Hot Country,"Today's top country hits of the week, worldwid...",53,6475778
3,37i9dQZF1DX10zKzsJ2jva,¡Viva Latino!,"Today's top Latin hits are right here, on ¡Viv...",50,11218997
4,37i9dQZF1DX4JAvHpjipBk,New Music Friday,"New music from Lorde, Young Thug, Ed Sheeran, ...",100,3788318
...,...,...,...,...,...
1390,37i9dQZF1DX3WvGXE8FqYX,Women of Pop,Kick back with the queens of pop.,70,2185426
1391,5ji4GZJpll6twskFvKxiHx,dw-c,NaN,50,14
1392,40VxbK9NqccdUDUpiUXmbp,dw_g,NaN,30,8
1393,0RTz1jFo5BXGPfI8eVf8sj,Top Shower Songs,NaN,100,57


In [9]:
pd_playlist.fillna('None')

,playlist_id,playlist_name,playlist_description,playlist_num_tracks,playlist_num_followers
0,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
1,37i9dQZF1DX0XUsuxWHRQd,RapCaviar,"New music from Young Thug, Trippie Redd, Rod W...",50,14030878
2,37i9dQZF1DX1lVhptIYRda,Hot Country,"Today's top country hits of the week, worldwid...",53,6475778
3,37i9dQZF1DX10zKzsJ2jva,¡Viva Latino!,"Today's top Latin hits are right here, on ¡Viv...",50,11218997
4,37i9dQZF1DX4JAvHpjipBk,New Music Friday,"New music from Lorde, Young Thug, Ed Sheeran, ...",100,3788318
...,...,...,...,...,...
1390,37i9dQZF1DX3WvGXE8FqYX,Women of Pop,Kick back with the queens of pop.,70,2185426
1391,5ji4GZJpll6twskFvKxiHx,dw-c,None,50,14
1392,40VxbK9NqccdUDUpiUXmbp,dw_g,None,30,8
1393,0RTz1jFo5BXGPfI8eVf8sj,Top Shower Songs,None,100,57


In [10]:
pd_playlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1395 entries, 0 to 1394
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   playlist_id             1395 non-null   object
 1   playlist_name           1395 non-null   object
 2   playlist_description    1239 non-null   object
 3   playlist_num_tracks     1395 non-null   int64 
 4   playlist_num_followers  1395 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 54.6+ KB


In [11]:
pd_artist

,id,name,followers,popularity,genres
0,2xgAJkalFHfceCNGETOkyM,GRX,76585,43,[]
1,6enyYil5r9snkIazgAX5DG,Jo Ann Kelly,998,20,['british blues']
2,40oYPr305MsT2lsiXr9fX9,The Boomtown Rats,116274,54,"['new romantic', 'new wave', 'new wave pop', '..."
3,3i8iJVU0mtgzbZsuF1AoJ3,sad alex,23548,52,"['alt z', 'modern indie pop']"
4,5oFkj1qSlyBUmV5d6Edgtq,Cozz,146831,57,['underground hip hop']
...,...,...,...,...,...
34649,1tY6Z4NEPI3xHhVA8VU9Jl,Vishwa Mohan Bhatt,23094,30,"['hindustani classical', 'hindustani instrumen..."
34650,7lPv78tlT005m22dYvSdNg,Lindsey Leblanc,52,32,[]
34651,6iXZPP5qLB2dtIQ4WuNk9M,Thelonious Coltrane,4359,44,"['chillhop', 'jazz boom bap', 'lo-fi jazzhop']"
34652,7s9LTAKRwkBQ0CqWlbE254,Frase,2245,36,[]


In [12]:
pd_artist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34654 entries, 0 to 34653
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          34654 non-null  object
 1   name        34654 non-null  object
 2   followers   34654 non-null  int64 
 3   popularity  34654 non-null  int64 
 4   genres      34654 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.3+ MB


## Mô tả dữ liệu

+ Dataframe `pd_track` mô tả thông tin về một playlist sẽ chứa các track nào, một `playlist_id` sẽ có thể có nhiều `track_id`. Ngoài ra còn một số thông tin khác ở đây chẳng hạn như độ phổ biến, thời lượng track,...
+ Sử dụng các `track_id` để truy xuất các thông tin đặc trưng về âm thanh tại dataframe `pd_audio`. Tại Dataframe này, các đặc trưng âm học sẽ được biểu diễn, đây sẽ là dataframe chúng ta sẽ thực hiện các công việc chuẩn hóa và tính toán độ dị biệt giữa các track

In [13]:
pd_track = pd.merge(pd_track, pd_audio, left_on='track_id', right_on='id')
pd_track

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity,id,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,37i9dQZF1DXcBWIGoYBM5M,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,...,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
1,37i9dQZF1DXcRXFNfZr7Tp,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,...,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
2,37i9dQZF1DXc2aPBXGmXrt,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,...,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
3,37i9dQZF1DWYD2pxgooM7y,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,...,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
4,37i9dQZF1DWUa8ZRTfalHk,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,...,1,-5.484,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88618,0RTz1jFo5BXGPfI8eVf8sj,3Zuf70897YkrVRAsrBMMSF,"You're The One That I Want - From ""Grease"" Ori...",168466,"['4hKkEHkaqCsyxNxXEsszVH', '4BoRxUdrcgbbq1rxJv...","['John Travolta', 'Olivia Newton-John']",0CbwlsDekgmAW03uMBjPBG,Grease (Deluxe Edition),0,3Zuf70897YkrVRAsrBMMSF,...,0,-8.198,1,0.0935,0.2270,0.000784,0.2090,0.774,106.976,4
88619,4lgseztVwmKQ8MNETPVIny,1MDoll6jK4rrk2BcFRP5i7,Hello,295502,['4dpARuHxo51G3z768sgnrY'],['Adele'],5y28RdPzIRXegzr5YM9Y0D,Hello,3,1MDoll6jK4rrk2BcFRP5i7,...,5,-6.129,0,0.0342,0.3290,0.000000,0.0854,0.289,157.980,4
88620,4lgseztVwmKQ8MNETPVIny,7DqTaelFf846rl1CHmyfOW,Chan Chan,257737,['11kBu957KTYoAltZHDm8gW'],['Buena Vista Social Club'],2Upqk0mMh9OMIVSj9F8Xzw,Buena Vista Social Club,0,7DqTaelFf846rl1CHmyfOW,...,2,-14.183,0,0.0345,0.8420,0.031100,0.1120,0.641,83.222,4
88621,4lgseztVwmKQ8MNETPVIny,0ygTmpa6uSotkBkTiwcMZ4,Warwick Avenue,226386,['37NqXwtb6nIEqRt4EJSoIO'],['Duffy'],4x0HJqUUDRvJLZbie95Ayk,Rockferry (Deluxe Edition),0,0ygTmpa6uSotkBkTiwcMZ4,...,0,-5.814,0,0.0392,0.3390,0.000000,0.1210,0.502,169.908,4


`pd_full` sẽ là một Dataframe tổng hợp tất cả thông tin bao gồm cả các đặc trưng về audio

In [14]:
pd_full = pd.merge(pd_track, pd_playlist, left_on='playlist_id', right_on='playlist_id')
pd.set_option('display.max_columns', None)
pd_full

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,playlist_name,playlist_description,playlist_num_tracks,playlist_num_followers
0,37i9dQZF1DXcBWIGoYBM5M,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,5HCyWlXZPP0y6Gqq8TgA20,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,4,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
1,37i9dQZF1DXcBWIGoYBM5M,27NovPIUIRrOZoCHxABJwK,INDUSTRY BABY (feat. Jack Harlow),212000,"['7jVv8c5Fj3E9VhNjxT4snq', '2LIk90788K0zvyj2JJ...","['Lil Nas X', 'Jack Harlow']",622NFw5Yk0OReMJ2XWcXUh,INDUSTRY BABY (feat. Jack Harlow),97,27NovPIUIRrOZoCHxABJwK,0.736,0.704,3,-7.409,0,0.0615,0.020300,0.000000,0.0501,0.894,149.995,4,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
2,37i9dQZF1DXcBWIGoYBM5M,3Vi5XqYrmQgOYBajMWSvCi,Need To Know,210560,['5cj0lLjcoR7YOSnhnX0Po5'],['Doja Cat'],1nAQbHeOWTfQzbOoFrvndW,Planet Her,92,3Vi5XqYrmQgOYBajMWSvCi,0.664,0.609,1,-6.509,1,0.0707,0.304000,0.000000,0.0926,0.194,130.041,4,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
3,37i9dQZF1DXcBWIGoYBM5M,6PQ88X9TkUIAUIZJHW2upE,Bad Habits,231041,['6eUKZXaKkcviH0Ku9w2n3V'],['Ed Sheeran'],01NhUvviMytvV12pmJiDZH,Bad Habits,98,6PQ88X9TkUIAUIZJHW2upE,0.808,0.897,11,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,4,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
4,37i9dQZF1DXcBWIGoYBM5M,3Wrjm47oTz2sjIgck11l5e,Beggin',211560,['0lAWpj5szCSwM4rUMHYmrr'],['Måneskin'],2qJw6w5XwQO0PQlSWPu7Tw,Chosen,99,3Wrjm47oTz2sjIgck11l5e,0.714,0.800,11,-4.808,0,0.0504,0.127000,0.000000,0.3590,0.589,134.002,4,Today's Top Hits,Doja Cat is on top of the Hottest 50!,50,29379379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88618,3yUO32lGCEnAfFGORSEsy4,6jKNYORXeWGGGiSkj1sBZu,Cocky Tuff Up,170111,['5mYWkDD4b1eM4ZjFq5axxs'],['Aidonia'],4iSfhNdg4CcSoHBZ8MNR0S,Matte Black Riddim,0,6jKNYORXeWGGGiSkj1sBZu,0.776,0.924,7,-4.651,1,0.0580,0.243000,0.000075,0.1580,0.749,104.975,4,Raw Expression,Brukout edition: It's a raw and sexy dancehall...,42,307
88619,3yUO32lGCEnAfFGORSEsy4,5DycXVa4e82E6k9M15Nf51,Turn It Up (feat. DJ Jazzy T),220978,"['1RnGhd2JfN5nbVOvYmhDyO', '4DxQlFDzfkQjyU9mrq...","[""Ce'Cile"", 'DJ Jazzy T']",0ZEmsBG911uKT7CJxlF0DS,Still Running,6,5DycXVa4e82E6k9M15Nf51,0.646,0.947,7,-3.631,1,0.0772,0.000389,0.000057,0.8960,0.793,100.027,4,Raw Expression,Brukout edition: It's a raw and sexy dancehall...,42,307
88620,3yUO32lGCEnAfFGORSEsy4,41gA2hLHRJW4bC40lm2X7R,Cup Yah Full,227552,['21J3YJTyq1biE3SvSNjzuf'],['Capleton'],1GIhVkizUAMw7Zci6DMPdG,Cup Yah Full - Single,0,41gA2hLHRJW4bC40lm2X7R,0.507,0.858,5,-6.156,0,0.2850,0.234000,0.000000,0.1450,0.591,97.682,4,Raw Expression,Brukout edition: It's a raw and sexy dancehall...,42,307
88621,3yUO32lGCEnAfFGORSEsy4,0K22864F8jwAymy5JsopR9,Touch Yah,156708,['06JH3wcXkJg7eoxZVMO1UX'],['T.O.K'],2HOMRFOtWZrMdHkGqyuQJR,D-Stress Riddim,0,0K22864F8jwAymy5JsopR9,0.634,0.811,0,-4.661,0,0.3370,0.399000,0.000000,0.3030,0.794,99.644,4,Raw Expression,Brukout edition: It's a raw and sexy dancehall...,42,307


In [15]:
pd_full = pd_full.drop('playlist_name', axis=1)
pd_full = pd_full.drop('playlist_description', axis=1)
pd_full = pd_full.drop('id', axis=1)

In [16]:
pd_full

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,playlist_num_tracks,playlist_num_followers
0,37i9dQZF1DXcBWIGoYBM5M,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,4,50,29379379
1,37i9dQZF1DXcBWIGoYBM5M,27NovPIUIRrOZoCHxABJwK,INDUSTRY BABY (feat. Jack Harlow),212000,"['7jVv8c5Fj3E9VhNjxT4snq', '2LIk90788K0zvyj2JJ...","['Lil Nas X', 'Jack Harlow']",622NFw5Yk0OReMJ2XWcXUh,INDUSTRY BABY (feat. Jack Harlow),97,0.736,0.704,3,-7.409,0,0.0615,0.020300,0.000000,0.0501,0.894,149.995,4,50,29379379
2,37i9dQZF1DXcBWIGoYBM5M,3Vi5XqYrmQgOYBajMWSvCi,Need To Know,210560,['5cj0lLjcoR7YOSnhnX0Po5'],['Doja Cat'],1nAQbHeOWTfQzbOoFrvndW,Planet Her,92,0.664,0.609,1,-6.509,1,0.0707,0.304000,0.000000,0.0926,0.194,130.041,4,50,29379379
3,37i9dQZF1DXcBWIGoYBM5M,6PQ88X9TkUIAUIZJHW2upE,Bad Habits,231041,['6eUKZXaKkcviH0Ku9w2n3V'],['Ed Sheeran'],01NhUvviMytvV12pmJiDZH,Bad Habits,98,0.808,0.897,11,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,4,50,29379379
4,37i9dQZF1DXcBWIGoYBM5M,3Wrjm47oTz2sjIgck11l5e,Beggin',211560,['0lAWpj5szCSwM4rUMHYmrr'],['Måneskin'],2qJw6w5XwQO0PQlSWPu7Tw,Chosen,99,0.714,0.800,11,-4.808,0,0.0504,0.127000,0.000000,0.3590,0.589,134.002,4,50,29379379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88618,3yUO32lGCEnAfFGORSEsy4,6jKNYORXeWGGGiSkj1sBZu,Cocky Tuff Up,170111,['5mYWkDD4b1eM4ZjFq5axxs'],['Aidonia'],4iSfhNdg4CcSoHBZ8MNR0S,Matte Black Riddim,0,0.776,0.924,7,-4.651,1,0.0580,0.243000,0.000075,0.1580,0.749,104.975,4,42,307
88619,3yUO32lGCEnAfFGORSEsy4,5DycXVa4e82E6k9M15Nf51,Turn It Up (feat. DJ Jazzy T),220978,"['1RnGhd2JfN5nbVOvYmhDyO', '4DxQlFDzfkQjyU9mrq...","[""Ce'Cile"", 'DJ Jazzy T']",0ZEmsBG911uKT7CJxlF0DS,Still Running,6,0.646,0.947,7,-3.631,1,0.0772,0.000389,0.000057,0.8960,0.793,100.027,4,42,307
88620,3yUO32lGCEnAfFGORSEsy4,41gA2hLHRJW4bC40lm2X7R,Cup Yah Full,227552,['21J3YJTyq1biE3SvSNjzuf'],['Capleton'],1GIhVkizUAMw7Zci6DMPdG,Cup Yah Full - Single,0,0.507,0.858,5,-6.156,0,0.2850,0.234000,0.000000,0.1450,0.591,97.682,4,42,307
88621,3yUO32lGCEnAfFGORSEsy4,0K22864F8jwAymy5JsopR9,Touch Yah,156708,['06JH3wcXkJg7eoxZVMO1UX'],['T.O.K'],2HOMRFOtWZrMdHkGqyuQJR,D-Stress Riddim,0,0.634,0.811,0,-4.661,0,0.3370,0.399000,0.000000,0.3030,0.794,99.644,4,42,307


In [17]:
pd_full = pd_full.drop_duplicates()
pd_full = pd_full.fillna('None')
pd_full

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,playlist_num_tracks,playlist_num_followers
0,37i9dQZF1DXcBWIGoYBM5M,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,4,50,29379379
1,37i9dQZF1DXcBWIGoYBM5M,27NovPIUIRrOZoCHxABJwK,INDUSTRY BABY (feat. Jack Harlow),212000,"['7jVv8c5Fj3E9VhNjxT4snq', '2LIk90788K0zvyj2JJ...","['Lil Nas X', 'Jack Harlow']",622NFw5Yk0OReMJ2XWcXUh,INDUSTRY BABY (feat. Jack Harlow),97,0.736,0.704,3,-7.409,0,0.0615,0.020300,0.000000,0.0501,0.894,149.995,4,50,29379379
2,37i9dQZF1DXcBWIGoYBM5M,3Vi5XqYrmQgOYBajMWSvCi,Need To Know,210560,['5cj0lLjcoR7YOSnhnX0Po5'],['Doja Cat'],1nAQbHeOWTfQzbOoFrvndW,Planet Her,92,0.664,0.609,1,-6.509,1,0.0707,0.304000,0.000000,0.0926,0.194,130.041,4,50,29379379
3,37i9dQZF1DXcBWIGoYBM5M,6PQ88X9TkUIAUIZJHW2upE,Bad Habits,231041,['6eUKZXaKkcviH0Ku9w2n3V'],['Ed Sheeran'],01NhUvviMytvV12pmJiDZH,Bad Habits,98,0.808,0.897,11,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,4,50,29379379
4,37i9dQZF1DXcBWIGoYBM5M,3Wrjm47oTz2sjIgck11l5e,Beggin',211560,['0lAWpj5szCSwM4rUMHYmrr'],['Måneskin'],2qJw6w5XwQO0PQlSWPu7Tw,Chosen,99,0.714,0.800,11,-4.808,0,0.0504,0.127000,0.000000,0.3590,0.589,134.002,4,50,29379379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88618,3yUO32lGCEnAfFGORSEsy4,6jKNYORXeWGGGiSkj1sBZu,Cocky Tuff Up,170111,['5mYWkDD4b1eM4ZjFq5axxs'],['Aidonia'],4iSfhNdg4CcSoHBZ8MNR0S,Matte Black Riddim,0,0.776,0.924,7,-4.651,1,0.0580,0.243000,0.000075,0.1580,0.749,104.975,4,42,307
88619,3yUO32lGCEnAfFGORSEsy4,5DycXVa4e82E6k9M15Nf51,Turn It Up (feat. DJ Jazzy T),220978,"['1RnGhd2JfN5nbVOvYmhDyO', '4DxQlFDzfkQjyU9mrq...","[""Ce'Cile"", 'DJ Jazzy T']",0ZEmsBG911uKT7CJxlF0DS,Still Running,6,0.646,0.947,7,-3.631,1,0.0772,0.000389,0.000057,0.8960,0.793,100.027,4,42,307
88620,3yUO32lGCEnAfFGORSEsy4,41gA2hLHRJW4bC40lm2X7R,Cup Yah Full,227552,['21J3YJTyq1biE3SvSNjzuf'],['Capleton'],1GIhVkizUAMw7Zci6DMPdG,Cup Yah Full - Single,0,0.507,0.858,5,-6.156,0,0.2850,0.234000,0.000000,0.1450,0.591,97.682,4,42,307
88621,3yUO32lGCEnAfFGORSEsy4,0K22864F8jwAymy5JsopR9,Touch Yah,156708,['06JH3wcXkJg7eoxZVMO1UX'],['T.O.K'],2HOMRFOtWZrMdHkGqyuQJR,D-Stress Riddim,0,0.634,0.811,0,-4.661,0,0.3370,0.399000,0.000000,0.3030,0.794,99.644,4,42,307


## Ý tưởng thuật toán
+ Tính K tracks trong tập hợp tất cả các tracks (trừ N seed tracks) có khoảng cách L2 gần nhất với từng seed track, ta được tập N x K tracks.
+ Trả về các bài hát 

## Xây dựng dữ liệu để test
+ Chọn ngẫu nhiên 100 `playlist_id` để tạo một dictionary trong đó key là `playlist_id` và value là list chứa các `track_id` có trong playlist đó
+ Xây dựng tập test bằng cách bỏ đi một vài `track_id` trong list của `playlist_id` dựa vào `missing_rate`

`playlist_test_all_track` là một dictionary chứa một list sau khi được filter ra, mang ý nghĩa playlist này bao gồm tất cả các track trong playlist đó

In [18]:
playlist_test_size = 100

playlist_test_all_track = generate_playlist(pd_full, pd_playlist, num_playlist_to_test=playlist_test_size)
playlist_test_all_track

{'37i9dQZF1DWX16Q183eyvd': ['0weAUscowxeqDtpCgtbpgp',
  '3QpAo1aiTUVgsQBR8KmNHO',
  '1JIICzNjNibNpB3HeefvFm',
  '6ZSMbNa2ZSmLR7LlC5Bz0m',
  '00Nx5kFUJBcdGa0GNPH8Lx',
  '6l8PVMHkToebBTlJacW6Fa',
  '3vyExFn4NbOCGkGR30i6AA',
  '0PylWiOSendDTztmimb42y',
  '2b9sTGqIWVTFVu7AUMQn7h',
  '3xDiCJdE8eRRgx0qUOQphX',
  '0GxUrzpQT8TR76G8ijZyTb',
  '7nzvfB4c708km3Y69qKj5k',
  '0ajAIY2IEOjJ4JFmlAEuvM',
  '3Ti0GdlrotgwsAVBBugv0I',
  '56LCZNVDZBgVXmsP37ksgy',
  '5AEyhmFToITN1n2Wa2zLHI',
  '1CoEBzmNnIJlNRrgcGZ1Lz',
  '5oGi3t0P5LJ4Opu87mlLUg',
  '1iD60ApOiYKOb32UCaYWP1',
  '1o7pq0fvaVfQSKOKwQAHSo',
  '51VcV1Wqh57DzVa8Tew2qs',
  '4IrDPdjKLcQNnCoP4vWF6o',
  '0qaUiV6xlTRYqQZ2332Qsh',
  '57rPxsDFpBnUp4Bw5uxIXP',
  '60kcJIrsehemBC5S2PTTG3',
  '6c3ipL19bLPHrpoj5kKawy',
  '3MZmpfcmRK73emDMcqNpbe',
  '5GBH7uQ6hX7umS4Bl9RZ8J',
  '7j0tMRulMtwmNpyOZmHtD0',
  '7ECdYi17tlPuGg9YVblRPU',
  '5GJkaptoXzXzZ3NUw8MJy9',
  '4lWJMQSOJIWSodi3Oe6Few',
  '7JaMhrgXmPSXDjVAVxrDav',
  '2t3ZxwOLtpRYxctPbXRRns',
  '4UAdHXcCnQFmdqaRUsn

Do chúng ta không có dữ liệu test nên chúng ta bắt buộc phải phát sinh dữ liệu để đo chất lượng của mô hình

`playlist_to_test` là một dictionary tương tự như `playlist_test_all_track`, chỉ là `playlist_to_test` đã bỏ đi một số lượng các track trong playlist đó (số lượng phụ thuộc vào `missing_rate`)

In [19]:
missing_rate = 0.2
playlist_to_test = generate_test_playlist(playlist_test_all_track, missing_rate)
playlist_to_test

{'37i9dQZF1DWX16Q183eyvd': ['0weAUscowxeqDtpCgtbpgp',
  '3QpAo1aiTUVgsQBR8KmNHO',
  '1JIICzNjNibNpB3HeefvFm',
  '6ZSMbNa2ZSmLR7LlC5Bz0m',
  '00Nx5kFUJBcdGa0GNPH8Lx',
  '6l8PVMHkToebBTlJacW6Fa',
  '3vyExFn4NbOCGkGR30i6AA',
  '0PylWiOSendDTztmimb42y',
  '2b9sTGqIWVTFVu7AUMQn7h',
  '3xDiCJdE8eRRgx0qUOQphX',
  '0GxUrzpQT8TR76G8ijZyTb',
  '7nzvfB4c708km3Y69qKj5k',
  '0ajAIY2IEOjJ4JFmlAEuvM',
  '3Ti0GdlrotgwsAVBBugv0I',
  '56LCZNVDZBgVXmsP37ksgy',
  '5AEyhmFToITN1n2Wa2zLHI',
  '1CoEBzmNnIJlNRrgcGZ1Lz',
  '5oGi3t0P5LJ4Opu87mlLUg',
  '1iD60ApOiYKOb32UCaYWP1',
  '1o7pq0fvaVfQSKOKwQAHSo',
  '51VcV1Wqh57DzVa8Tew2qs',
  '4IrDPdjKLcQNnCoP4vWF6o',
  '0qaUiV6xlTRYqQZ2332Qsh',
  '57rPxsDFpBnUp4Bw5uxIXP',
  '60kcJIrsehemBC5S2PTTG3',
  '6c3ipL19bLPHrpoj5kKawy',
  '3MZmpfcmRK73emDMcqNpbe',
  '5GBH7uQ6hX7umS4Bl9RZ8J',
  '7j0tMRulMtwmNpyOZmHtD0',
  '7ECdYi17tlPuGg9YVblRPU',
  '5GJkaptoXzXzZ3NUw8MJy9',
  '4lWJMQSOJIWSodi3Oe6Few',
  '7JaMhrgXmPSXDjVAVxrDav',
  '2t3ZxwOLtpRYxctPbXRRns',
  '4UAdHXcCnQFmdqaRUsn

`pd_full_test` là Dataframe được xây dựng nên sau khi lấy ra từ các `playlist_all_id_to_test`, chứa tất cả các track trong playlist, đây là Dataframe chính để lưu các thông tin truy suất cho khó có thể cho người khác hiểu bằng các id

`pd_full_test_copy` sẽ được sử dụng chỉ giữ lại các cột đặc trưng audio để tiến hành tính toán

In [20]:
pd_full_test = pd_full.copy()
playlist_all_id_to_test = list(playlist_to_test.keys())
pd_full_test = pd_full_test[pd_full_test['playlist_id'].isin(playlist_all_id_to_test)]
pd_full_test = pd_full_test.reset_index(drop=True)
pd_full_test_copy = pd_full_test.copy()
pd_full_test

,playlist_id,track_id,track_name,track_duration,artist_ids,artist_names,album_id,album_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,playlist_num_tracks,playlist_num_followers
0,37i9dQZF1DWVlLVXKTOAYa,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),141805,"['2tIP7SsRs7vjIcLrU85W8J', '1uNFoZAHBGtllmzznp...","['The Kid LAROI', 'Justin Bieber']",4QLAtpLNUsHEYrcHXmMIZZ,STAY (with Justin Bieber),100,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,4,100,917164
1,37i9dQZF1DWVlLVXKTOAYa,6PQ88X9TkUIAUIZJHW2upE,Bad Habits,231041,['6eUKZXaKkcviH0Ku9w2n3V'],['Ed Sheeran'],01NhUvviMytvV12pmJiDZH,Bad Habits,98,0.808,0.897,11,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,4,100,917164
2,37i9dQZF1DWVlLVXKTOAYa,6PERP62TejQjgHu81OHxgM,good 4 u,178147,['1McMsnEElThX1knmY4oliG'],['Olivia Rodrigo'],3rMjL8NA5Wh2hbMNk2fSlY,good 4 u,75,0.556,0.661,6,-5.052,0,0.2040,0.300000,0.000000,0.1010,0.668,168.560,4,100,917164
3,37i9dQZF1DWVlLVXKTOAYa,748mdHapucXQri7IAO8yFK,Kiss Me More (feat. SZA),208866,"['5cj0lLjcoR7YOSnhnX0Po5', '7tYKF4w9nC0nq9CsPZ...","['Doja Cat', 'SZA']",1OnzqJTL9bwe4kvaLxRYxt,Kiss Me More (feat. SZA),94,0.762,0.701,8,-3.541,1,0.0286,0.235000,0.000158,0.1230,0.742,110.968,4,100,917164
4,37i9dQZF1DWVlLVXKTOAYa,463CkQjx2Zk1yXoBuierM9,Levitating (feat. DaBaby),203064,"['6M2wZ9GZgrQXHCFfjv46we', '4r63FhuTkUYltbVAg5...","['Dua Lipa', 'DaBaby']",04m06KhJUuwe1Q487puIud,Levitating (feat. DaBaby),88,0.702,0.825,6,-3.787,0,0.0601,0.008830,0.000000,0.0674,0.915,102.977,4,100,917164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1m6DKwKNM1YLcm3OX6RzJg,30YuvcPzM120mgeNchky6p,Say it Like You Mean it,196500,['44pv88VpkRu39v0lupR2gZ'],['Maria Hazell'],38cgcDlZXNWPgU1L4OLbhf,Say it Like You Mean it,12,0.485,0.832,0,-4.471,0,0.1320,0.015100,0.005630,0.2590,0.280,159.885,5,63,3348
7191,1m6DKwKNM1YLcm3OX6RzJg,0Wm74MutbYsqdNk5DRqPX8,Inte OK,193958,['53mirS5AzwzEw9MsquLXFo'],['Min Stora Sorg'],5lTCy0H7QkUCWsilyhkhfq,"Mvh Anso, M$$",1,0.486,0.190,2,-11.718,1,0.0274,0.819000,0.000014,0.3070,0.176,83.083,4,63,3348
7192,1m6DKwKNM1YLcm3OX6RzJg,3WZFBxHWyIVlzH4nDAyR0B,Wild At Heart,191200,['1mDQs9Mv3bVIJJyx2XCawe'],['Jaquline Ronneklew'],0RY8mgqyjUUfXRa47kf9FF,Wild At Heart,0,0.473,0.737,11,-6.636,0,0.0464,0.067800,0.000005,0.2680,0.358,120.299,4,63,3348
7193,1m6DKwKNM1YLcm3OX6RzJg,4tews9FvmahNNBMbgtEx4s,Let's Get Fxxxed One Last Time,267341,['5PjNp1KvPD5VXn7nj0xSMc'],['Gutarra'],3ICZKkjv6P9K5YlfMFrRtf,Let's Get Fxxxed One Last Time,2,0.415,0.839,9,-5.447,1,0.0521,0.000679,0.000001,0.1110,0.324,157.978,4,63,3348


In [21]:
columns_to_drop = ['playlist_id', 'track_id', 'track_name', 'artist_ids', 'artist_names', 'album_id', 'album_name',
                   'playlist_num_tracks', 'playlist_num_followers']
pd_full_test_copy = pd_full_test_copy.drop(columns_to_drop, axis=1)
pd_full_test_copy

,track_duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,141805,100,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,4
1,231041,98,0.808,0.897,11,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,4
2,178147,75,0.556,0.661,6,-5.052,0,0.2040,0.300000,0.000000,0.1010,0.668,168.560,4
3,208866,94,0.762,0.701,8,-3.541,1,0.0286,0.235000,0.000158,0.1230,0.742,110.968,4
4,203064,88,0.702,0.825,6,-3.787,0,0.0601,0.008830,0.000000,0.0674,0.915,102.977,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,196500,12,0.485,0.832,0,-4.471,0,0.1320,0.015100,0.005630,0.2590,0.280,159.885,5
7191,193958,1,0.486,0.190,2,-11.718,1,0.0274,0.819000,0.000014,0.3070,0.176,83.083,4
7192,191200,0,0.473,0.737,11,-6.636,0,0.0464,0.067800,0.000005,0.2680,0.358,120.299,4
7193,267341,2,0.415,0.839,9,-5.447,1,0.0521,0.000679,0.000001,0.1110,0.324,157.978,4


In [22]:
sum = 0
for key in playlist_test_all_track.keys():
    sum += len(playlist_test_all_track[key])
assert sum == pd_full_test_copy.shape[0]

In [ ]:
pd_full_test_copy.info()

# Tiền xử lí dữ liệu

## Tiến hành chuẩn hóa và xây dựng mô hình
+ Sử dụng MinMaxScaler() để chuẩn hóa dữ liệu
+ Sử dụng mô hình K-NearestNeighbor để tiến hành đề xuất các bài hát lân cận
+ Số lượng k không quan trọng trong thuật toán này, nếu số lượng k càng nhiều thì càng có một danh sách bài hát lớn hơn để đề xuất. Do đó ta sẽ sử dụng siêu tham số `n_preds` trong hàm đề xuất để quyết định

In [24]:
scaler = MinMaxScaler()
scaler.fit(pd_full_test_copy)
scaled_pd_full_test = scaler.transform(pd_full_test_copy)

In [25]:
nn = NearestNeighbors(n_neighbors=10).fit(scaled_pd_full_test)
distances, idx = nn.kneighbors(scaled_pd_full_test)
distances

array([[0.        , 0.1       , 0.1       , ..., 0.3007607 , 0.3007607 ,
        0.3007607 ],
       [0.        , 0.        , 0.        , ..., 0.34398493, 0.34398493,
        0.35158857],
       [0.        , 0.30310931, 0.32131912, ..., 0.35200195, 0.35712945,
        0.35828046],
       ...,
       [0.        , 0.24195172, 0.24263188, ..., 0.26807687, 0.26957698,
        0.27363755],
       [0.        , 0.14604166, 0.20687468, ..., 0.26598882, 0.27285152,
        0.27597394],
       [0.        , 0.16716419, 0.16948196, ..., 0.23732732, 0.24008541,
        0.24320192]])

In [26]:
idx

array([[   0,  175,  134, ...,  402,  106,    5],
       [ 152,    1,  200, ...,  218,  368,  690],
       [   2,   15, 5471, ...,  996, 3197,  428],
       ...,
       [7192, 2724,  563, ..., 2262, 4471,  801],
       [7193, 3486, 2699, ..., 3546,  578, 1433],
       [7194, 2535,  790, ..., 2160,  725,  707]], dtype=int64)

Cột distance đầu có vẻ như là các bài hát được cover hoặc các bài hát được thực hiện bởi nhiều ca sĩ khác nhau nên không có nhiều sự sai khác, do đó chúng ta sẽ quan tâm tới cột thứ hai trở đi

Ngoài ra chúng ta cũng nên bỏ qua các trường hợp khoảng cách ra 0 vì nhiều khi các bài hát giống nhau có ID khác nhau là một điều dễ gặp trên Spotify

In [27]:
pd_full_test_copy

,track_duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,141805,100,0.591,0.764,1,-5.484,1,0.0483,0.038300,0.000000,0.1030,0.478,169.928,4
1,231041,98,0.808,0.897,11,-3.712,0,0.0348,0.046900,0.000031,0.3640,0.591,126.026,4
2,178147,75,0.556,0.661,6,-5.052,0,0.2040,0.300000,0.000000,0.1010,0.668,168.560,4
3,208866,94,0.762,0.701,8,-3.541,1,0.0286,0.235000,0.000158,0.1230,0.742,110.968,4
4,203064,88,0.702,0.825,6,-3.787,0,0.0601,0.008830,0.000000,0.0674,0.915,102.977,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,196500,12,0.485,0.832,0,-4.471,0,0.1320,0.015100,0.005630,0.2590,0.280,159.885,5
7191,193958,1,0.486,0.190,2,-11.718,1,0.0274,0.819000,0.000014,0.3070,0.176,83.083,4
7192,191200,0,0.473,0.737,11,-6.636,0,0.0464,0.067800,0.000005,0.2680,0.358,120.299,4
7193,267341,2,0.415,0.839,9,-5.447,1,0.0521,0.000679,0.000001,0.1110,0.324,157.978,4


## Với một playlist bất kỳ

`playlist_all_id_to_test` chứa tất cả các `playlist_id` của các playlist được chọn ra 

`playlist_id_to_test` là một `playlist_id` lấy ngẫu nhiên để kiểm tra

In [28]:
class ColDropper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns_to_drop = ['playlist_id', 'track_id',
                                'track_name', 'artist_ids',
                                'artist_names', 'album_id',
                                'album_name', 'playlist_num_tracks',
                                'playlist_num_followers']

    def fit(self, x, y=None):
        return self

    def transform(self, x, y=None):
        result = x.copy()
        result = result.drop(columns=self.columns_to_drop)
        return result

In [29]:
full_pipeline = Pipeline([('ColDropper', ColDropper()), ('Scaler', MinMaxScaler()), ('KNN', NearestNeighbors())])


In [30]:
print(full_pipeline.get_params().keys())

dict_keys(['memory', 'steps', 'verbose', 'ColDropper', 'Scaler', 'KNN', 'Scaler__clip', 'Scaler__copy', 'Scaler__feature_range', 'KNN__algorithm', 'KNN__leaf_size', 'KNN__metric', 'KNN__metric_params', 'KNN__n_jobs', 'KNN__n_neighbors', 'KNN__p', 'KNN__radius'])


In [31]:
playlist_id_to_test = playlist_all_id_to_test[0]
playlist_id_to_test

'37i9dQZF1DWX16Q183eyvd'

In [32]:
all_name_songs = list(
    pd_full[pd_full['track_id'].isin(playlist_test_all_track[playlist_id_to_test])]['track_name'].values)
print('All song in playlist 0:', all_name_songs)

All song in playlist 0: ["Gettin' Jiggy Wit It", "Gettin' Jiggy Wit It", 'Old Town Road', 'Girls vs. Boys', 'Dance Party in the House', 'Rise', 'Bored is a Bad Word', 'Bubble Wrap Rap', 'Dance to the Reggae Rhythm (feat. Father Goose)', 'Kool Kids', 'Wonderful People (feat. Judah Marley, Gideon Marley & Abraham Marley)', 'Girls Run Everything', 'Apple of My Eye', 'Hiphop-O-Potamus - Feat. Jabari & Baby Power', 'Can I Kick It?', 'Do My Dance', 'Take Me To Jamaica (feat. Toots Hibbert)', "If You're Happy and You Know It", 'Go Play Outside', 'Go DJ', 'Me Myself And I', 'Man Stop', 'TELL EM', 'Safari', 'Mi Gente', 'Getting Silly', 'Boomerang', 'Kids That Rock', 'Learn Your State Capitols', 'Walk Tall (feat. Paul Simon)', "I'm Healthy", 'In My Feelings', 'Three Little Birds (feat. Ziggy Marley)', 'This Old Man', 'Shake Ya Tailfeather', '1 Times Tables (TI Style)', '2 Times Tables (Biggie Style)', 'Cha Cha Slide', 'One Love / People Get Ready - Medley', 'I Like Fruit', 'Panini', 'Tennessee',

In [33]:
given_name_songs = list(pd_full[pd_full['track_id'].isin(playlist_to_test[playlist_id_to_test])]['track_name'].values)
print('Given in playlist 0 : ', given_name_songs)

Given in playlist 0 :  ["Gettin' Jiggy Wit It", "Gettin' Jiggy Wit It", 'Old Town Road', 'Girls vs. Boys', 'Dance Party in the House', 'Rise', 'Bored is a Bad Word', 'Bubble Wrap Rap', 'Dance to the Reggae Rhythm (feat. Father Goose)', 'Kool Kids', 'Wonderful People (feat. Judah Marley, Gideon Marley & Abraham Marley)', 'Girls Run Everything', 'Apple of My Eye', 'Hiphop-O-Potamus - Feat. Jabari & Baby Power', 'Can I Kick It?', 'Do My Dance', 'Take Me To Jamaica (feat. Toots Hibbert)', "If You're Happy and You Know It", 'Go Play Outside', 'Go DJ', 'Me Myself And I', 'Man Stop', 'TELL EM', 'Safari', 'Mi Gente', 'Getting Silly', 'Boomerang', 'Kids That Rock', 'Learn Your State Capitols', 'Walk Tall (feat. Paul Simon)', "I'm Healthy", 'In My Feelings', 'Three Little Birds (feat. Ziggy Marley)', 'This Old Man', 'Shake Ya Tailfeather', '1 Times Tables (TI Style)', '2 Times Tables (Biggie Style)', 'Cha Cha Slide', 'One Love / People Get Ready - Medley', 'I Like Fruit', 'Panini', 'Tennessee']


`pid` để lưu lại (các) `playlist_id` mà tại đó các `track_id` được cho xuất hiện nhiều nhất

`most_cluster` để lưu lại label mà tại đó có nhiều track nhất trong `all_name_songs` thuộc về label đó

việc tiếp theo chỉ là lấy ra các bài hát thuộc label đó trong `pd_full_test` mà khác các track được cho trước

# Mô hình hóa dữ liệu

In [34]:
import collections


def recommend_songs(given_playlist_track, pd_full_test, info_df, n_pred):
    pid = list(info_df[info_df['track_id'].isin(given_playlist_track)]['playlist_id'].value_counts().index)[0]

    full_pipeline.set_params(KNN__n_neighbors=n_pred)
    full_pipeline.fit(X=pd_full_test)

    pd_given_feature = info_df[info_df['track_id'].isin(given_playlist_track)]
    pd_given_feature = full_pipeline['ColDropper'].fit_transform(pd_given_feature)

    pd_given_feature = full_pipeline['Scaler'].fit_transform(pd_given_feature)

    distances, idx = full_pipeline['KNN'].kneighbors(pd_given_feature)
    idx = idx[:, 1:]
    distances = distances[:, 1:]
    idx = idx.flatten()

    counter = collections.Counter(idx)
    a = counter.most_common(len(counter))

    result = []
    track_duplicates = []
    i = 0
    k = 0

    while k != n_pred:
        track_id = info_df.loc[a[i][0], 'track_id']
        if (track_id not in given_playlist_track) and (track_id not in track_duplicates):
            track_duplicates.append(track_id)
            track_name = info_df.loc[a[i][0], 'track_name']
            result.append(track_name)
            i += 1
            k += 1
        else:
            i += 1

    return result

In [35]:
recommended_songs = recommend_songs(given_playlist_track=playlist_to_test[playlist_id_to_test],
                                    pd_full_test=pd_full_test,
                                    info_df=pd_full_test,
                                    n_pred=len(playlist_test_all_track[playlist_id_to_test]) - len(
                                        playlist_to_test[playlist_id_to_test]))

In [36]:
recommended_songs = recommend_songs(
    given_playlist_track=['5HCyWlXZPP0y6Gqq8TgA20', '4XvcHTUfIlWfyJTRG0aqlo', '27NovPIUIRrOZoCHxABJwK'],
    pd_full_test=pd_full_test,
    info_df=pd_full_test,
    n_pred=len(playlist_test_all_track[playlist_id_to_test]) - len(playlist_to_test[playlist_id_to_test]))

In [37]:
print(recommended_songs)

['She Works Hard For The Money', 'Ballroom Blitz', 'Get Lucky (feat. Pharrell Williams & Nile Rodgers)', 'Crocodile Rock', 'Do It Again', 'Listen to the Music', "That's All - Remastered 2007", 'Heart of Gold - 2009 Remaster', 'Aqualung', 'Champagne Supernova', 'With a Little Help from My Friends - Live From London']


## Đánh giá mô hình đối với các playlist được sinh ra

In [38]:
def test_multi(pd_playlist, pd_full, missing_rate=0.2, nums_playlists_test=100):
    r_score = []

    playlist_test_all_track = generate_playlist(pd_full, pd_playlist, num_playlist_to_test=nums_playlists_test)
    playlist_to_test = generate_test_playlist(playlist_test_all_track, missing_rate=missing_rate)

    all_playlistID_for_tests = list(playlist_to_test.keys())

    pd_full_tests = pd_full.copy()
    pd_full_tests = pd_full_tests[pd_full_tests['playlist_id'].isin(all_playlistID_for_tests)]
    pd_full_tests = pd_full_tests.reset_index(drop=True)

    pd_full_tests_copy = pd_full_tests.copy()

    recommended_songs_all_playlist = {}

    for each_playlist in all_playlistID_for_tests:
        recommended_songs = recommend_songs(playlist_to_test[each_playlist],
                                            pd_full_tests_copy,
                                            pd_full_tests,
                                            len(playlist_test_all_track[each_playlist]) - len(
                                                playlist_to_test[each_playlist]))
        recommended_songs_all_playlist[each_playlist] = recommended_songs
        all_name_songs = list(
            pd_full[pd_full['track_id'].isin(playlist_test_all_track[each_playlist])]['track_name'].values)
        given_name_songs = list(pd_full[pd_full['track_id'].isin(playlist_to_test[each_playlist])]['track_name'].values)
        songs_need_to_recommend = [song for song in all_name_songs if song not in given_name_songs]
        r_score.append(r_precision(recommended_songs, songs_need_to_recommend))
    return r_score

In [39]:
r_score = test_multi(pd_playlist, pd_full, missing_rate=0.2, nums_playlists_test=100)


In [40]:
missing_rates = np.arange(0.1, 1, 0.1)
for missing_rate in missing_rates:
    r_score = test_multi(pd_playlist, pd_full, missing_rate=missing_rate, nums_playlists_test=100)
    print('*' * 50)
    print('Mising rate = ', missing_rate)
    print('Average R-Precision', np.array(r_score).mean())
    print('Max R-Precision', np.array(r_score).max())
    print('*' * 50)

**************************************************
Mising rate =  0.1
Average R-Precision 0.016166666666666666
Max R-Precision 0.5
**************************************************
**************************************************
Mising rate =  0.2
Average R-Precision 0.02489596819589079
Max R-Precision 0.4
**************************************************
**************************************************
Mising rate =  0.30000000000000004
Average R-Precision 0.034785569862071083
Max R-Precision 0.5333333333333333
**************************************************
**************************************************
Mising rate =  0.4
Average R-Precision 0.03935821640327218
Max R-Precision 0.675
**************************************************
**************************************************
Mising rate =  0.5
Average R-Precision 0.04167531924248978
Max R-Precision 0.5
**************************************************
**************************************************
Mising ra